In [1]:
from dotenv import load_dotenv
load_dotenv()

# import os
# os.environ["OPENAI_API_KEY"]

True

In [2]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
import json
import pandas as pd

In [3]:
model = ChatOpenAI(
    # model='gpt-3.5-turbo-16k-0613',
    model='gpt-3.5-turbo-1106',
    temperature = 0.1
)

In [4]:
prompt1 = ChatPromptTemplate.from_template("""
from the given social media posts from people in the given province. please analize base on un sdg goals. return output only in json format

### province
ปทุมธานี

### social media posts
ปทุมธานีรถติดมาก
อุบัติเหตุหน้ามธ.รถติดยาว
มธ.เดินทางไปอย่างไรจากสระบุรี

### instruction
1. identify user group that has posted on social media
2. as a psychologist, please analyze did the users have posted those massageas 
3. classify sentiment of the post whether positive, neutal or negative. If negative following step 4, 5 and 6 otherwise go to step 5 and 6.
4. identify a concern or an issue
5. identify a wish of the users
6. return output in json format that contain

### example province
{example_province}

### example social media posts
{example_social_posts}

### example output format
{example_output}

### province
{province}

### social media posts
{social_posts}

### output format
""")

# chain1 = prompt1 | model | StrOutputParser()


In [6]:
chain1_example={
    "province":"ปทุมธานี",
    "social_posts":["ไม่มีสานสาธารณะดีๆให้เราเลย","ต้นไม้ยืนต้นตายริมถนนอีกแล้ว"],
    "output":{
      "user_group": "Local residents and commuters in Pathum Thani Province",
      "psychological_analysis": "The users are expressing their frustrations and challenges related to transportation. This indicates a concern for personal efficiency, daily commuting stress, and a desire for better traffic management.",
      "sentiment_classification": "negative",
      "identified_concerns": {
        "traffic_congestion": "High levels of traffic congestion",
        "road_safety": "Concerns about road safety due to accidents"
      },
      "user_wishes": [
        "improved_traffic_management: Desire for better traffic flow and management",
        "safer_road_conditions: Wish for safer road conditions and fewer accidents"
      ],
      "relevance_sdg_target": [
        "SDG_11.2 By 2030, provide access to safe, affordable, accessible and sustainable transport systems for all",
        "SDG_3.6 By 2020, halve the number of global deaths and injuries from road traffic accidents"
      ]
    }
}

post_data={
    'example_province':chain1_example['province'],
    'example_social_posts': '\n'.join(chain1_example['social_posts']),
    'example_output':chain1_example['output'],
    'province':"ปทุมธานี",
    'social_posts':'\n'.join(["ไม่มีสานสาธารณะดีๆให้เราเลย","ต้นไม้ยืนต้นตายริมถนนอีกแล้ว"])
}
# social_analysis=chain1.invoke(post_data)
# obj_social_analysis=json.loads(social_analysis)
# print(social_analysis)
# targets=[i.split(' ')[0][4:] for i in obj_social_analysis['relevance_sdg_target']]
# print(targets)

In [7]:
from langchain.schema import BaseOutputParser

class IndicatorOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, social_analysis: dict):
        obj_social_analysis=json.loads(social_analysis)
        targets=[i.split(' ')[0][4:] for i in obj_social_analysis['relevance_sdg_target']]
        df2=pd.read_csv('sdg.csv')
        df2=df2.query("target == @targets")
        text_indicators=[]
        for idx,i in df2[['indicator','indicator_description']].iterrows():
            text_indicators.append({'indicator':i['indicator'], 'indicator_description':i['indicator_description']}  )
        obj_social_analysis['relevance_sdg_indicators']=text_indicators
        return obj_social_analysis

# print( IndicatorOutputParser().parse(social_analysis))

In [8]:
chain1 = prompt1 | model |  StrOutputParser() | IndicatorOutputParser()
# type(chain1.invoke(post_data))

In [9]:
chain2_example ={
    "social_analysis":{
      "user_group": "Local residents in Pathum Thani Province",
      "psychological_analysis": "The users are expressing disappointment and concern about the lack of public green spaces and the deterioration of roadside trees. This indicates a concern for environmental quality and a desire for better public green spaces and tree conservation.",
      "sentiment_classification": "negative",
      "identified_concerns": {
        "lack_of_public_green_spaces": "Disappointment over the absence of good public green spaces",
        "deteriorating_roadside_trees": "Concern about the continuous death of roadside trees"
      },
      "user_wishes": [
        "improved_public_green_spaces: Desire for better public green spaces",
        "tree_conservation: Wish for the conservation and care of roadside trees"
      ],
      "relevance_sdg_target": [
        "SDG_11.7 By 2030, provide universal access to safe, inclusive and accessible, green and public spaces, in particular for women and children, older persons and persons with disabilities",
        "SDG_15.2 By 2020, promote the implementation of sustainable management of all types of forests, halt deforestation, restore degraded forests and substantially increase afforestation and reforestation globally"
      ],
      "relevance_sdg_indicators": [
          {'indicator': '11.7.1', 'indicator_description': 'Average share of the built-up area of cities that is open space for public use for all, by sex, age, and persons with disabilities'},
          {'indicator': '11.7.2', 'indicator_description': 'Proportion of persons victim of physical or sexual harassment, by sex, age, disability status, and place of occurrence in the previous 12 months'},
          {'indicator': '15.2.1', 'indicator_description': 'Progress towards sustainable forest management'},
          {'indicator': '15.2.2', 'indicator_description': 'Net forest conversion rate'},
          {'indicator': '15.2.3', 'indicator_description': 'Proportion of important sites for terrestrial and freshwater biodiversity that are covered by protected areas, by ecosystem type'}
      ]
    },    
    "output":{
      # "challenges": [
      #   "Lack of accessible public green spaces in urban areas, leading to reduced quality of life and environmental health.",
      #   "Significant loss of trees along roadsides, impacting local biodiversity, aesthetics, and air quality."
      # ],
        "challenge_topic": "Uban Green Space",
        "challenge_description": "Lack of accessible public green spaces in urban areas, leading to reduced quality of life and environmental health.",
      "indicators": [
        "SDG_Indicator_11.7.1: This indicator reflects the need for more public green spaces in urban areas, aligning with the community's desire for accessible green spaces.",
        "SDG_Indicator_15.2.1: Progress towards sustainable forest management is crucial for addressing the loss of roadside trees and promoting reforestation."
      ],
      "solutions": [
        "Develop and implement urban planning policies that prioritize the creation and maintenance of public green spaces, with a focus on inclusivity and accessibility for all demographic groups.",
        "Initiate local tree planting and preservation campaigns, involving community members in the planting and care of trees, thus fostering a sense of ownership and responsibility towards local greenery.",
        "Collaborate with environmental experts to identify and use tree species that are native to the area, more resilient, and require less maintenance, ensuring long-term sustainability.",
        "Incorporate green spaces and tree preservation into school curricula and community programs to raise awareness and educate the public about the importance of urban greenery and sustainable environmental practices."
      ]
    }
}


In [10]:
prompt2 = ChatPromptTemplate.from_template("""
Form social post anylysis and the given sdg indicators, please anaylize challenges 
then select indicator and provide posible solutions 
that can fulfil the gap between user wishes and the indicator in the style of Bjorn Lomborg. 
return output only in json format

### example social post anylysis
{example_chain2_social_analysis}

### example output json format
{example_chain2_output}

### social post anylysis
{obj_social_analysis}

### output json format
""")

chain2 = (
    {
        "obj_social_analysis": chain1, 
        "example_chain2_social_analysis": itemgetter("example_chain2_social_analysis"),
        "example_chain2_output": itemgetter("example_chain2_output")
    }
    | prompt2
    | model
    | StrOutputParser()
)


post_data={
    'example_province':chain1_example['province'],
    'example_social_posts': '\n'.join(chain1_example['social_posts']),
    'example_output':chain1_example['output'],
    'province':"ปทุมธานี",
    'social_posts':'\n'.join(["เด็กเล็กในชุมชนสลัมไม่มีที่วิ่งเล่นต้องมาเล่นบนท้องถนน","ชุมชนสลัมมีพื้นเพจากเพชรบูรณ์"]),
    'example_chain2_social_analysis':chain2_example['social_analysis'],
    'example_chain2_output':chain2_example['output'],
}


In [11]:
c1=chain1.invoke(post_data)
c1

{'user_group': 'Local residents in Pathum Thani Province',
 'psychological_analysis': 'The users are expressing concern for the safety and well-being of children in their community, as they do not have a safe place to play. This indicates a concern for child safety and the lack of recreational spaces.',
 'sentiment_classification': 'negative',
 'identified_concerns': {'child_safety': 'Lack of safe play areas for children',
  'community_infrastructure': 'Inadequate recreational spaces in the community'},
 'user_wishes': ['safe_play_areas: Wish for safe and designated play areas for children',
  'improved_community_infrastructure: Desire for better community infrastructure and recreational spaces'],
 'relevance_sdg_target': ['SDG_11.7 By 2030, provide universal access to safe, inclusive and accessible, green and public spaces, in particular for women and children, older persons and persons with disabilities',
  'SDG_3.2 By 2030, end preventable deaths of newborns and children under 5 yea

In [12]:
c2=chain2.invoke(post_data)
c2

'{\n  "challenge_topic": "Child Safety and Recreational Spaces",\n  "challenge_description": "Lack of safe and designated play areas for children, leading to concerns for child safety and well-being in the community.",\n  "indicators": [\n    "SDG_Indicator_11.7.1: This indicator reflects the need for safe and inclusive recreational spaces for children, aligning with the community\'s desire for designated play areas.",\n    "SDG_Indicator_3.2.1: The under-5 mortality rate is impacted by the availability of safe play areas and recreational spaces for children, highlighting the importance of addressing this issue."\n  ],\n  "solutions": [\n    "Collaborate with local authorities to designate and develop safe play areas within the community, ensuring accessibility and inclusivity for all children, including those with disabilities.",\n    "Implement community-led initiatives to create and maintain recreational spaces, involving parents, educators, and children in the planning and design p

In [13]:
# Here's another example, but with a compound typed field.
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
from typing import List
from langchain.output_parsers import OutputFixingParser

class Challenge(BaseModel):
    challenge_description: str = Field(description="a gap between the people wishes and the current situation")
    challenge_topic: str = Field(description="short name of the challenge")
    indicators: List[str] = Field(description="list of relevance SDG indicators")
    solutions: List[str] = Field(description="list of solutions for the challenges")


actor_query = "Extract information from the social post analysis"

parser = PydanticOutputParser(pydantic_object=Challenge)

prompt = PromptTemplate(
    template="Answer the query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# _input = prompt.format_prompt(query=actor_query)

# output = model(_input.to_string())

# parser.parse(c2)

##new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())
new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

chain3 = (
    {
        "obj_social_analysis": chain1, 
        "example_chain2_social_analysis": itemgetter("example_chain2_social_analysis"),
        "example_chain2_output": itemgetter("example_chain2_output")
    }
    | prompt2
    | model
    | StrOutputParser()
    |new_parser
)
c3=chain3.invoke(post_data)

In [14]:
c3

Challenge(challenge_description='Lack of safe and designated play areas for children, leading to concerns for child safety and well-being in the community.', challenge_topic='Child Safety and Recreational Spaces', indicators=["SDG_Indicator_11.7.1: This indicator reflects the need for safe and inclusive recreational spaces for children, aligning with the community's desire for designated play areas.", 'SDG_Indicator_3.2.1: The under-5 mortality rate is impacted by the availability of safe play areas and recreational spaces for children, highlighting the importance of addressing this issue.'], solutions=['Collaborate with local authorities to designate and develop safe play areas within the community, ensuring accessibility and inclusivity for all children, including those with disabilities.', 'Implement community-led initiatives to create and maintain recreational spaces, involving parents, caregivers, and children in the planning and design process.', 'Advocate for policies that prior

In [15]:
c3.challenge_topic

'Child Safety and Recreational Spaces'

In [16]:
c3.challenge_description

'Lack of safe and designated play areas for children, leading to concerns for child safety and well-being in the community.'

In [17]:
c3.indicators

["SDG_Indicator_11.7.1: This indicator reflects the need for safe and inclusive recreational spaces for children, aligning with the community's desire for designated play areas.",
 'SDG_Indicator_3.2.1: The under-5 mortality rate is impacted by the availability of safe play areas and recreational spaces for children, highlighting the importance of addressing this issue.']

In [18]:
c3.solutions

['Collaborate with local authorities to designate and develop safe play areas within the community, ensuring accessibility and inclusivity for all children, including those with disabilities.',
 'Implement community-led initiatives to create and maintain recreational spaces, involving parents, caregivers, and children in the planning and design process.',
 'Advocate for policies that prioritize child-friendly urban planning, integrating safe play areas into the development of public spaces and infrastructure.',
 'Provide educational programs and resources to raise awareness about the importance of safe play areas for children and the impact on child well-being and mortality rates.']

### Unit Test

In [19]:
post_data={
        'example_province':chain1_example['province'],
        'example_social_posts': '\n'.join(chain1_example['social_posts']),
        'example_output':chain1_example['output'],
        'province':"ปทุมธานี",
        'social_posts':'\n'.join(["ประเทศไทย","นักเรียนจบแล้วมีทักษะไม่พร้อมสำหรับการทำงาน"]),
        'example_chain2_social_analysis':chain2_example['social_analysis'],
        'example_chain2_output':chain2_example['output'],
    }
with open('template.json', 'w') as f:
    json.dump(post_data, f)

In [21]:
with open('template.json', 'r') as openfile:
    template = json.load(openfile)
c3=chain3.invoke(post_data)
c3

Challenge(challenge_description='Concerns about the readiness and skills of recent graduates for the workforce, leading to difficulties in transitioning from education to employment.', challenge_topic='Workforce Readiness and Skills Gap', indicators=["SDG_Indicator_4.4.1: This indicator reflects the need for relevant skills, including technical and vocational skills, aligning with the community's desire for improved education quality and skills development.", 'SDG_Indicator_8.6.1: The proportion of youth not in education, employment, or training highlights the need for support in transitioning from education to employment, addressing the concerns expressed by the user group.'], solutions=['Enhance collaboration between educational institutions and industries to align curriculum with the skills needed in the workforce, ensuring graduates are equipped with relevant and practical skills.', 'Establish mentorship and internship programs to provide hands-on experience and guidance for studen

In [ ]:
### pack into a function

In [22]:
def get_challenge_solutions(location="ประเทศไทย",posts=["นักเรียนจบแล้วมีทักษะไม่พร้อมสำหรับการทำงาน",]):
    with open('template.json', 'r') as openfile:
        template = json.load(openfile)
    template['province']=location
    template['social_posts']=posts
    
    c3=chain3.invoke(template)
    d={}
    d['challenge_topic']=c3.challenge_topic
    d['challenge_description']=c3.challenge_description
    d['indicators']=c3.indicators
    d['solutions']=c3.solutions
    return d
get_challenge_solutions()

{'challenge_topic': 'Job Readiness for Recent Graduates',
 'challenge_description': 'Lack of job readiness and skills among recent graduates, leading to concerns about employment and entrepreneurship opportunities.',
 'indicators': ['SDG_Indicator_4.4.1: The proportion of youth and adults with information and communications technology (ICT) skills reflects the need for relevant job skills among recent graduates.'],
 'solutions': ['Enhance technical and vocational training programs to include ICT skills and other relevant job skills that are in demand in the current job market.',
  'Establish partnerships between educational institutions and industry stakeholders to ensure that the curriculum is aligned with the needs of the workforce, providing practical training and internships opportunities.',
  'Create mentorship and career counseling programs to guide recent graduates in developing the necessary skills and knowledge for successful employment and entrepreneurship.',
  'Promote entre

In [24]:
get_challenge_solutions('ลำปาง',['ลำปางอากาศร้อยมาก'])

{'challenge_topic': 'Climate Resilience',
 'challenge_description': 'The need to strengthen resilience and adaptive capacity to climate-related hazards and natural disasters in Lampang Province.',
 'indicators': ['SDG_Indicator_13.1.1: The number of deaths, missing persons, and directly affected persons attributed to disasters per 100,000 population is a key indicator for assessing the impact of climate-related hazards and natural disasters on the community.'],
 'solutions': ['Implement and enforce building codes and land use regulations that consider climate-related hazards and natural disaster risks, ensuring the safety and resilience of infrastructure and communities.',
  'Develop and implement early warning systems and emergency response plans that are accessible and inclusive for all residents and visitors, enabling timely and effective responses to climate-related hazards and natural disasters.',
  'Invest in infrastructure and technology that can mitigate the impact of climate-r

In [26]:
get_challenge_solutions('นครศรีธรรมราช',"""
ตามหาโทรศัพท์หายครับ รุ่น realme C17 สีน้ำเงิน ใส่เคสใสมีรูปนักเรียนอยู่หลังเคส ตกหล่นที่ถนนสายท่าซัก-ปากคร ช่วงเวลาประมาณ 17.00 น.
หากใครพบเจอรบกวนช่วยติดต่อมาหน่อยนะครับ
โทรไปหลายสายตั้งแต่ช่วงเย็นโทรติดแต่ไม่มีคนรับครับ และเมื่อเวลาประมาณ 19.00 น. โทรศัพท์ได้โดนปิดเครื่องไปแล้วครับ
#หากใครพบเจอติดต่อทางเบอร์ 👉🏻0952837915👈🏻
#ขอบคุณครับ🙏🏻
""")

{'challenge_topic': 'Lost Phone and Lack of Response',
 'challenge_description': 'Distress and frustration due to the loss of a phone and lack of response, leading to a sense of helplessness and desire for assistance and support.',
 'indicators': [],
 'solutions': ['Encourage the use of phone tracking apps and services to aid in the recovery of lost phones.',
  'Establish community support networks or online platforms where individuals can seek assistance and advice in finding lost items, including phones.',
  'Promote the importance of backing up data and securing personal devices to mitigate the impact of lost phones.',
  'Educate the community on the proper steps to take when a phone is lost, including reporting to authorities and utilizing available resources for assistance.']}

In [27]:
get_challenge_solutions('ปทุมธานี',"""
ผมลืมทำ กะเป๋าที่ตู้ฝากเงิน กสิกร  ข้าง7-11
ตลาด ลาดสวาย ครับ
ท่านได พบเห็น T  0927595133  ขอบัตร ทำงาน คืน ครับ  ขอบคุณ ครับ
""")

{'challenge_topic': 'Lost Wallet and Assistance',
 'challenge_description': 'Difficulty in retrieving a lost wallet and seeking assistance with card cancellation, leading to frustration and inconvenience for the individual.',
 'indicators': [],
 'solutions': ['Utilize local community forums and social media platforms to seek assistance from fellow residents or visitors who may have found the lost wallet.',
  'Contact local authorities and businesses in the specific location where the wallet was lost to inquire about any found items or assistance with card cancellation.',
  'Raise awareness about the importance of returning lost items and providing assistance to those in need within the community, fostering a culture of mutual support and cooperation.']}

In [28]:
get_challenge_solutions('พระนครศรีอยุธยา',"""
เรื่องเกิดขึ้นร้อนๆค่ะ ด้วยความคับข้องใจและเสียผลประโยชน์
วันนี้ 16.09.23 เวลาประมาณ 10.15 ไปเปิดกล้องวงจรที่ร้านดูได้เลยค่ะ ตั้งจะจะนำพ้อนในบัตรเครดิต KTC ไปแลกกาฟากกินฉ่ำๆๆ 399 แต้ม
ไปถึงที่ร้านก็บอกพนักงานว่า แลกคะแนน KTC ค่ะ ก่อนก้มหน้า ก้มตาเข้าแอพเพื่อกดคูปองที่แลกไว้ให้พนักงานสแกน
ระหว่างที่กำลังเข้าแอพพนักงานแคชเชียร์หันไปถามอีกคนว่าแลกคะแนนยังไง อีกคนบอกว่าที่ร้านไม่ได้รับแลก
แต่!!!ชั้นเห็นสแตนตั้งหน้าเคสเตอร์ ของแต่ละโปรโมชั่นที่รับแลกกกกก ค่ายซิมมือต่างๆนานา และหนึ่งในนั้นคือ KTC

และชั้นก็กดรับคูปองเตรียมให้สแกน(เป็นการยืนยันใช้คะแนนเรียบร้อย)
สรุป แลกไม่ได้ ไม่รับแลก ด้วยเหตุอะไรไม่ทราบ พนักงานพร้อมเสนอให้ไปอีกสาขานึง
คูปองเมื่อกดแล้วอยู่ได้ 15นาทีจ้าาาาาาาา ไปไม่ทันไงคะะะะ
เสียคะแนนฟรีๆ399คะแนน ก็อยากให้พันธ์ไทยชี้แจงทีค่ะ แล้วก็ช่วยไปเทรนพนักงานให้มีความรู้มากพอด้วย
😡😡
""")

{'challenge_topic': 'Customer Service and Trust in Brands',
 'challenge_description': 'Poor customer service and lack of trust in local businesses, leading to negative consumer experiences and potential disengagement from the market.',
 'indicators': [],
 'solutions': ['Implement customer service training programs for store staff, focusing on effective communication, problem-solving, and empathy towards customers.',
  'Establish feedback mechanisms for customers to provide input on their experiences, allowing businesses to address concerns and improve service quality.',
  'Promote transparency and accountability in business practices, building trust and confidence among consumers through clear and ethical communication.',
  'Encourage local businesses to participate in community engagement activities and initiatives, fostering a sense of connection and responsibility towards their customers and the community as a whole.']}

In [29]:
get_challenge_solutions('นครพนม',"""
จขกท มีทริปไปเที่ยงเชียงรายปลายเมษานี้ กับ นครพนมสกลนครมิถุนายนค่ะ

เชียงราย พักในเมืองทุกวัน ตั้งใจไม่ไปค้างดอยค่ะ
วันที่ 1 ศุกร์ ลงเครื่อง 12.40 รับรถเช่า ไปวัดร่องเสือเต้น วัดห้วยปลากั้ง วัดร่องขุ่น จบยามเย็นที่สิงห์ปาร์ค เข้าที่พัก
วันที่ 2 เสาร์ ครึ่งเช้าไปไร่ชา 101 ดอยแม่สลอง ครึ่งบ่ายไร่ชาฉุยฟง กลับลงมาเข้าที่พัก
วันที่ 3 อาทิตย์ ยังไม่มีโปรแกรมค่ะ 
วันที่ 4 จันทร์ ยังไม่มีโปรแกรมค่ะ  เครื่องออก 13.10 

อยากขอคำแนะนำสถานที่เที่ยวในวันที่ 3 และ 4 รวมถึงร้านอาหารที่ต้องไปทานค่ะ

นครพนมสกลนคร 
วันที่ 1 พฤหัส เครื่องลงนครพนม 09.10 รับรถเช่า ตีไปวัดพระธาตุท่าอุเทน วัดธาตุประสิทธิ์ เข้าสกลนคร วัดพระธาตุรารายณ์เจงเวง วัดอัมลการาม ปู่องค์ดำ แวะเก็บของที่โรงแรมในเมือง (ไปต่อที่ร้านครามสกล วัดถ้ำผาแด่น แล้วกลับมาพระธาตุเชิงชุมค่ะ) ในวงเล็บถ้าเหนื่อยจะเก็บไว้ไปวันรุ่งขึ้นค่ะ
วันที่ 2 ศุกร์ (ร้านครามสกล วัดถ้ำผาแด่น แล้วกลับมาพระธาตุเชิงชุม) เข้านครพนม วัดพระธาตุศรีคุณ วัดพระธาตุพนม วัดธาตุเรณู วัดมรุกขนคร เข้าโรงแรม เย็นถนนคนเดิน
วันที่ 3 เสาร์ พระธาตุนคร วัดธาตุมหาชัย 
วันที่ 4 กลับ กทม เครื่องออก 09.40 คงไม่ได้ทำอะไรค่ะ

วันที่ 3 ดูจะว่างนาน มีโปรแกรมอะไรสามารถทำได้บ้างคะ ชี้เป้าร้านอาหารนครพนมสกลนครให้ด้วยนะคะ
""")

{'challenge_topic': 'Enhancing Travel Experience',
 'challenge_description': 'Desire for a fulfilling and enjoyable travel experience among local residents and tourists, indicating a positive outlook and anticipation for travel plans.',
 'indicators': ['SDG_Indicator_3.4.1: The desire for a fulfilling travel experience aligns with the SDG target to promote mental health and well-being, as it reflects the importance of positive experiences and reducing stress-related health issues.',
  'SDG_Indicator_8.9.1: The emphasis on creating a fulfilling travel experience is linked to the promotion of sustainable tourism and its contribution to local economic growth.'],
 'solutions': ['Promote local cultural experiences and products to enhance the authenticity and enjoyment of the travel experience, supporting sustainable tourism and local economic development.',
  'Provide accessible information and resources for tourists to engage in physical activities and outdoor experiences, promoting mental

In [30]:
get_challenge_solutions('แม่ฮ่องสอน',"""
แพลนจะไป 3 ที่ ปางอุ๋ง บ้านรักไทย แล้วก็ ถ้ำน้ำลอด ปางมะผ้า 
ทั้ง 3 ที่นี้มีรถสาธารณะบริการเข้าถึงไหมครับ 
จากตัวเมืองแม่ฮ่องสอน ไป ปางอุ๋ง กับ บ้านรักไทย 
ออกมาแล้วขึ้นรถต่อไป บขส ปาย จะมีรถพาเข้าถ้ำไหม ขอบคุณครับ
""")

{'challenge_topic': 'Accessibility of Public Transportation in Remote Areas',
 'challenge_description': 'Limited availability of convenient and accessible public transportation options in remote areas, hindering the mobility and accessibility of local residents and tourists.',
 'indicators': ["SDG_Indicator_11.2.1: The indicator directly aligns with the community's wish for accessible public transportation, reflecting the need to improve convenient access to public transport for all demographic groups."],
 'solutions': ['Invest in the expansion and improvement of public transportation infrastructure in remote areas, ensuring that all communities have convenient access to reliable transport options.',
  'Implement community-based transportation initiatives, such as carpooling or shuttle services, to address the specific mobility needs of residents and tourists in remote areas.',
  'Collaborate with local authorities and transportation providers to conduct regular assessments of transpor